# Process Management

You can look at a process’ state by reading

    /proc/<PID>/status | grep State
    
Replace <PID> with the process ID (or self)

Results: 

- R: Running and runnable [Running and Waiting]
- S: Interruptible sleep [Blocked]
- D: Uninterruptible sleep [Blocked] (waiting for IO, network, etc., such that we cannot "wake it up" before the wait finishes)
- T: Stopped
- Z: Zombie

#### *If there are parents and child relationship between every single processes, how does the first one starts?*

> After the kernel initializes, it creates a single process from a program (pid = 1)

This process is called `init`, and it looks for it in `/sbin/init`  
Responsible for executing every other process on the machine --> This is ultimate ancestor  
Must always be active, if it exits the kernel thinks you’re shutting down

For Linux, `init` will probably be systemd but there’s other options

Aside: some operating systems create an “idle” process that the scheduler can run

![Alt text](images/image7.png)

- `systemd --user` is what we're using (user interface)
- `gnome-terminal-server` is if we open a terminal (the graphical stuffs, translating what we type, showing what we type, etc.)
- And then, the shell that executes on the terminal will be `zsh`

(notice: each browser tab is a process, so that a crashed tab will not affect others)


You can see your process tree with:

    htop

*(You can press F5 to switch between tree and list view)*

### Processes are assigned a Process ID (pid) on creation, and does not change throughout its lifetime

The process ID is just a number, and is unique for every **active** process  
On most Linux systems the maximum pid 32768, and <mark>0 is reserved (invalid)</mark>

Eventually the kernel will recycle a pid, after the process dies, for a new process

Remember: each process has its own address space (independent virtual memory)

***
### *What happends if the parent process exits (dies) before the child process dies??*

**There's a step between when the process exits and when the OS actually removes it**

- The operating system sets the exit status when a process terminates (the process terminates by calling `exit`)  
    - The process's memory is released
    - <mark>The process's PCB cannot be removed yet</mark>, so that the exit status of that process can still be read by its parent
- The minimum acknowledgment the parent has to do is read the child’s exit status
- Then, and only then, the process (child) can be entirely removed by the OS

There’s two situations:
1. The child exits first (zombie process)

> Zombie process: 'alive' and 'dead' at the same time, meaning it's not running anymore but the OS cannot completely get rid of the process yet (not acknowledged yet)

2. The parent exits first (orphan process)

> Orphan process: what the child process will become when its parent exit first

(and the parent will also need to be acknowledged by its parent)

How does this *acknowledgement* works?

--> <mark>Parent call `wait` to acknowledge</mark>

`wait` as the following API: (called within the parent process, takes no child pid argument, simply will wait and return to check if this process have any dying child process)

    pid_t wait(int status);

- status: Address to store the wait status of the process (passing an `int`, which will be modified)
- Returns the process ID of child process (whose death is being waited for)
    - -1: on failure (e.g. no children died)
    - 0: for non blocking calls with no child changes
    - \>0: the child with a change
    
The wait status contains a bunch of information, including the exit code  
Use `man wait` to find all the macros to query wait status  
You can use <mark>`waitpid`</mark> to wait on a specific child process

### Example 1: Parent blocks until the <mark>FIRST</mark> Child Process Exists, and cleans up

In [ ]:
%%file run.c

#include <errno.h>
#include <stdio.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <unistd.h>

int main(void) {
  pid_t pid = fork();
  if (pid == -1) {
    return errno;
  }
  if (pid == 0) {
    sleep(2);
  }
  else {
    printf("Calling wait\n");
    int wstatus;
    pid_t wait_pid = wait(&wstatus); // This will not return (BLOCKED) until the FIRST child process has exited
    if (WIFEXITED(wstatus)) {
      printf("Wait returned for an exited process! pid: %d, status: %d\n", wait_pid, WEXITSTATUS(wstatus));
    }
    else {
      return ECHILD;
    }
  }
  return 0;
}

Overwriting run.c


### Zombie Process waits for its Parent to Read its Exit Status

The process is terminated, but it hasn't been acknowledged

A process (parent) may have an error in it, where it never reads the child's exit status  
**--> The OS can interrupt the parent process to acknowledge the child (next lecture)**  
(a basic form of IPC - <mark>signal</mark>)

The OS has to keep a zombie process until it's acknowledged. IF the parent ignores and fails to acknowledge it, the zombie process needs to wait to be re-parented (--> becomes an **orphan**)

### Zombie process Example

In [ ]:
%%file run.c

#include <errno.h>
#include <fcntl.h>
#include <stdio.h>
#include <string.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

// This function prints the state of a process
int print_state(pid_t pid) {
  char buffer[4096];
  int ret = snprintf(buffer, sizeof(buffer), "/proc/%d/status", pid);
  if (ret < 0) {
    return ret;
  }
  int fd = open(buffer, O_RDONLY);
  if (fd < 0) {
    printf("unknown (macOS?)\n");
    return fd;
  }
  ret = read(fd, buffer, sizeof(buffer));
  if (ret < 0) {
    return ret;
  }
  close(fd);

  ssize_t bytes_read = ret;
  char state_prefix[] = "State:\t";
  for (int i = 0; i < bytes_read; ++i) {
    size_t remaining = bytes_read - i;
    size_t state_len = sizeof(state_prefix) - 1;
    if (remaining < state_len) {
      break;
    }
    if (strncmp(buffer + i, state_prefix, state_len) == 0) {
      for (int j = (i + state_len); j < bytes_read; ++j) {
	      ret = printf("%c", buffer[j]);
	      if (ret < 0) {
	        return ret;
	      }
	      if (buffer[j] == '\n') {
	        break;
	      }
      }
      break;
    }
  }

  return 0;
}

int main(void) {
  pid_t pid = fork();
  if (pid == -1) {
    return errno;
  }
  if (pid == 0) { // Child goes here
    sleep(2); // Sleeps the child process
  }
  else {  // Parent goes here
    int ret;
    sleep(1);
    printf("Child process state: ");
    ret = print_state(pid);    // This will print S (sleeping)
    if (ret < 0) { return errno; }
    sleep(2);
    printf("Child process state: ");
    ret = print_state(pid);   // This will print Z (zombie) --> Because the child process has exited and we NEVER called wait()
    if (ret < 0) { return errno; }
  }
  return 0;
}

### An Orphan Process Needs a New Parent

If the child process lost its parent process (either parent process finishes first, or parent fails to acknowledge its exit), the child still needs a process to acknowledge its exit

<mark>The OS re-parents the child process to `init` (pid = 1), which is now responsible to acknowledge the child</mark>

--> `init` does 2 jobs only: create processes, and call waitpid (to wait for its child (the actual child/assigned orphans)) to die

In [ ]:
%%file run.c

int main(int argc, char *argv[]) {
    pid_t pid = fork();
    if (pid == -1) {
        int err = errno;
        perror("fork failed");
        return err;
    }
    if (pid == 0) { // Child goes here
        printf("Child parent pid: %d\n", getppid());        // (*_*) This will always print the initial parent's pid 
        sleep(2);                                           // Initial parent is dead after this line --> orphan.
        printf("Child parent pid (after sleep): %d\n", getppid());  // Should print 1 (init process)
    }
    else {  // Parent goes here
        sleep(1);   // Say the parent process runs first, then during this 1 second, the child process will be ran anyways (by other threads or the same threads because CPU is free) --> (*_*)
    }
    return 0;
}



### Multiple fork example

In [ ]:
%%file run.c

#include <errno.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <unistd.h>

int new_process(int id) {
    pid_t pid = fork();
    if (pid > 0) {
        return 0;
    }
    else if (pid == -1) {
        int err = errno;
        perror("fork failed");
        return err;
    }

    for (int i = 0; i < 10; ++i) {
        printf("Process %d: %d\n", id, i);
        usleep(1000);
    }
    exit(0);
}

int main(void) {
  for (int i = 1; i <= 4; ++i) {
    int err = new_process(i);
    if (err != 0) {
        return err;
    }
  }
  return 0;
}

- We created 4 processes

- Are we a responsible parent? **NO**. We did not `wait()` for any child processes, meaning they'll all become orphans (after parent returns soon after each `return 0;`)